In [1]:
using Gtk

### The Following is the main menu

In [2]:
mainWin = GtkWindow("Home", 400, 400)
destroy(mainWin)
vboxMain = GtkBox(:v)
hboxMain1 = GtkBox(:h)
hboxMain2 = GtkBox(:h)
sentimentID = GtkButton("Sentiment Identifier")
similarPhrase = GtkButton("Find a similar phrase")
posVsNeg = GtkButton("Plot positive & negative words")
modelComparison = GtkButton("View model accuracy")
mainMenuLabel = GtkLabel("Main Menu")
push!(hboxMain1,sentimentID)
push!(hboxMain1,similarPhrase)
push!(hboxMain2, posVsNeg)
push!(hboxMain2, modelComparison)
push!(vboxMain, mainMenuLabel)
push!(vboxMain, hboxMain1)
push!(vboxMain, hboxMain2)
push!(mainWin, vboxMain)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Home", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=400, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE

### The Following is for the sentiment analysis feature

In [3]:
sentWin = GtkWindow("Sentiment Indentifier", 400,400)
destroy(sentWin)
prompt = GtkLabel("")
GAccessor.markup(prompt,"""<b>Please enter in a sentence or phrase that you wish to get the sentiment for  below.</b>\n""")
#User entry
ent = GtkEntry()
set_gtk_property!(ent,:text,"Enter text here")
str = get_gtk_property(ent,:text,String)
enter = GtkButton("Enter")
hbox = GtkBox(:h)
push!(hbox, ent)
push!(hbox, enter)

label = GtkLabel("Your text contains a ... sentiment.")
vbox = GtkBox(:v)
push!(vbox,prompt)
push!(vbox, hbox)
push!(vbox, label)
push!(sentWin, vbox)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Sentiment Indentifier", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=400, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-t

### The Following is for similar phrase feature

In [4]:
simPhraseWin = GtkWindow("Similar Phrase", 400, 400)
destroy(simPhraseWin)

### The Following is for ploting positive and negative words 

In [5]:
posNegWin = GtkWindow("Positve and Negative Word Plot", 400, 400)
destroy(posNegWin)

### The Following is for displaying model accuracy and comparions 

In [6]:
modelDisplayWin = GtkWindow("Display Model Accuracy", 400, 400)
destroy(modelDisplayWin)

### Functions for changing windows

In [7]:
function on_sentID_click(w)
    destroy(mainWin)
    showall(sentWin)
end
function on_simPhrase_click(w)
    destroy(mainWin)
    showall(simPhraseWin)
end
function on_PosNeg_click(w)
    destroy(mainWin)
    showall(posNegWin)
end
function on_modelDisp_click(w)
    destroy(mainWin)
    showall(modelDisplayWin)
end
signal_connect(on_sentID_click, sentimentID,"clicked")
signal_connect(on_simPhrase_click, similarPhrase,"clicked")
signal_connect(on_PosNeg_click, posVsNeg,"clicked")
signal_connect(on_modelDisp_click, modelComparison,"clicked")

0x000000da

In [8]:
showall(mainWin)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Home", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=400, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE,


(julia.exe:17228): Gtk-CRITICAL **: 14:21:24.421: gtk_widget_get_window: assertion 'GTK_IS_WIDGET (widget)' failed

(julia.exe:17228): Gtk-CRITICAL **: 14:21:24.421: gtk_widget_get_window: assertion 'GTK_IS_WIDGET (widget)' failed

(julia.exe:17228): Gtk-CRITICAL **: 14:21:24.421: gtk_widget_translate_coordinates: assertion 'GTK_IS_WIDGET (dest_widget)' failed

(julia.exe:17228): Gtk-CRITICAL **: 14:21:24.518: gtk_widget_get_window: assertion 'GTK_IS_WIDGET (widget)' failed

(julia.exe:17228): Gtk-CRITICAL **: 14:21:24.519: gtk_widget_get_window: assertion 'GTK_IS_WIDGET (widget)' failed

(julia.exe:17228): Gtk-CRITICAL **: 14:21:24.519: gtk_widget_translate_coordinates: assertion 'GTK_IS_WIDGET (dest_widget)' failed
